In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/All averages combined.csv")
df

Unnamed: 0  Disease  Limb Straightness Left hand  \
0    2_33_1.csv        1                     0.072180   
1    2_19_1.csv        1                     0.063107   
2    2_18_2.csv        1                     0.038577   
3    2_18_1.csv        1                     0.058668   
4    2_17_2.csv        1                     0.031050   
..          ...      ...                          ...   
315   5_7_3.csv        4                     0.129397   
316   5_8_1.csv        4                     0.078179   
317   5_8_2.csv        4                     0.089078   
318   5_8_3.csv        4                     0.081038   
319   5_9_1.csv        4                     0.132541   

     Limb Straightness Right hand  Limb Straightness Left leg  \
0                        0.039628                    0.046422   
1                        0.032303                    0.047958   
2                        0.041491                    0.059929   
3                        0.052902                    0.082212   
4                        0.045636                    0.060195   
..                            ...                         ...   
315                      0.103346                    0.071518   
316                      0.092183                    0.042246   
317                      0.097356                    0.055313   
318                      0.084248                    0.068758   
319                      0.135019                    0.032455   

     Limb Straightness Right leg  Right hand Left leg coordination  \
0                       0.036615                         16.496753   
1                       0.041224                         18.673947   
2                       0.036835                         14.712591   
3                       0.066757                         16.741215   
4                       0.036476                         14.374102   
..                           ...                               ...   
315                     0.082525                         18.275022   
316                     0.035168                         19.134692   
317                     0.044180                         27.963366   
318                     0.061518                         29.375185   
319                     0.057820                         14.691190   

     Left hand Right leg coordination  Upper Body Straightness  \
0                           23.911346                 0.420838   
1                           25.101639                 0.336320   
2                           17.828149                 0.138734   
3                           22.015808                 0.206714   
4                           16.793416                 0.394934   
..                                ...                      ...   
315                         23.142042                 0.389627   
316                         16.188418                 0.204224   
317                         20.325585                 0.317797   
318                         27.823953                 0.437682   
319                         15.475554                 0.402885   

     Body Straightness  ...  MD(12,13)  MD(12,14)  MD(12,15)  MD(12,16)  \
0             0.055522  ...   0.362899   0.045866   0.353619   0.157410   
1             0.060252  ...   0.372367   0.046454   0.344915   0.159672   
2             0.046092  ...   0.239068   0.045966   0.291759   0.101473   
3             0.056299  ...   0.174759   0.046175   0.257447   0.160351   
4             0.048441  ...   0.342950   0.047025   0.409245   0.151089   
..                 ...  ...        ...        ...        ...        ...   
315           0.123560  ...   0.246284   0.048671   0.291549   0.168892   
316           0.014547  ...   0.103133   0.048019   0.200357   0.167107   
317           0.030823  ...   0.115870   0.048129   0.202066   0.159378   
318           0.051921  ...   0.114563   0.049891   0.166820   0.124283   
319           0.164377  ...   0.268418   0.047882   0.279172   0.167666   

    

In [ ]:
print(df.columns)

Index(['Unnamed: 0', 'Disease', 'Limb Straightness Left hand',
       'Limb Straightness Right hand', 'Limb Straightness Left leg',
       'Limb Straightness Right leg', 'Right hand Left leg coordination',
       'Left hand Right leg coordination', 'Upper Body Straightness',
       'Body Straightness',
       ...
       'MD(12,13)', 'MD(12,14)', 'MD(12,15)', 'MD(12,16)', 'MD(13,14)',
       'MD(13,15)', 'MD(13,16)', 'MD(14,15)', 'MD(14,16)', 'MD(15,16)'],
      dtype='object', length=165)


In [ ]:
df=df.drop(['Unnamed: 0'],axis=1)
X = df.drop(['Disease'],axis=1)
y = df["Disease"]
# 1 for Hemiplegia
# 2 for Diplegia
# 3 for Normal
# 4 for parkinson

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# Generate a synthetic dataset for demonstration
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Define the grid of hyperparameters to search (for illustration purposes, use the best found previously)
best_params = {
    'n_estimators': 200,
    'learning_rate': 0.1,
    'max_depth': 4
}

# Apply the best hyperparameters to the model
xgb_model.set_params(**best_params)

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# List to store accuracy for each fold
fold_accuracies = []

# Perform 5-fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model
    xgb_model.fit(X_train_fold, y_train_fold)

    # Validate the model
    y_val_pred = xgb_model.predict(X_val_fold)
    val_accuracy = accuracy_score(y_val_fold, y_val_pred)
    fold_accuracies.append(val_accuracy)

    print(f"Fold validation accuracy: {val_accuracy}")

# Calculate and print the average accuracy across all folds
average_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print(f"Average cross-validation accuracy: {average_accuracy}")

# Train the model on the entire training set with the best hyperparameters
xgb_model.fit(X_train, y_train)

# Evaluate the model performance on the test set
test_accuracy = xgb_model.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

Fold validation accuracy: 0.9
Fold validation accuracy: 0.9
Fold validation accuracy: 0.89375
Fold validation accuracy: 0.90625
Fold validation accuracy: 0.925
Average cross-validation accuracy: 0.905
Test accuracy: 0.9
